In [7]:
import re

In [8]:
def parse_part(p: str) -> dict[str,int]:
    regex = re.compile(r"\{x=([0-9]+),m=([0-9]+),a=([0-9]+),s=([0-9]+)\}")
    m = regex.match(p)
    g = m.groups()
    return {'x': int(g[0]), 'm': int(g[1]), 'a': int(g[2]), 's': int(g[3])}

In [9]:
parse_part(r"{x=787,m=2655,a=1222,s=2876}")

{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}

In [10]:
def parse_wf(wf: str) -> dict[str, list[tuple[None|tuple[str, str, int], str]]]:
    re1 = re.compile(r"([a-z]+)\{(([xmas][<>][0-9]+:[a-zAR]+,)+)([a-zAR]+)\}")
    m1 = re1.match(wf)
    g1 = m1.groups()
    tag = g1[0]
    default = g1[-1]
    conditionals = g1[1]
    results = []
    re2 = re.compile(r"([xmas])([<>])([0-9]+):([a-zAR]+),?")
    m2 = re2.findall(conditionals)
    for m in m2:
        results.append(((m[0], 2*int(m[1]=='>')-1, int(m[2])) , m[3]))
    results.append((None, default))
    return {tag: results}

In [11]:
parse_wf("qqz{s>2770:qs,m<1801:hdj,R}")

{'qqz': [(('s', 1, 2770), 'qs'), (('m', -1, 1801), 'hdj'), (None, 'R')]}

In [12]:
def follow_wf(part: dict[str, int], tag: str, wf: dict[str, list[tuple[None|tuple[str, str, int], str]]], debug = False) -> int:
    conds = wf[tag]
    if debug:
        print(tag)
    for c in conds:
        if c[0] is None or (part[c[0][0]] - c[0][2])*c[0][1] > 0:
            if c[1] == 'R':
                return 0
            elif c[1] == 'A':
                return sum(x for x in part.values())
            else:
                return follow_wf(part, c[1], wf, debug)

In [13]:
with open('test.txt', 'rt') as f:
    test = f.read()

def parse_input(text: str):
    wf, parts = text.split('\n\n')
    wf_dict = {}
    for w in wf.split('\n'):
        wf_dict |= parse_wf(w)
    p = [parse_part(i.strip()) for i in parts.strip().split('\n')]
    return wf_dict, p

In [14]:
wf, parts = parse_input(test)

In [15]:
follow_wf(parts[0], 'in', wf, True)

in
qqz
qs
lnx


7540

In [16]:
follow_wf(parts[1], 'in', wf, True)

in
px
rfg
gd


0

In [17]:
follow_wf(parts[2], 'in', wf, True)

in
qqz
hdj
pv


4623

In [18]:
def part1(text: str) -> int:
    wf, parts = parse_input(text)
    return sum(follow_wf(p, 'in', wf) for p in parts)

In [19]:
part1(test)

19114

In [20]:
with open('input', 'rt') as f:
    inp = f.read()

In [21]:
with open('output1', 'wt') as f:
    f.write(str(part1(inp)))

In [52]:
def vol(cube: dict[str: tuple[int, int]]) -> int:
    return (cube['x'][1]-cube['x'][0]+1)*(cube['m'][1]-cube['m'][0]+1)*(cube['a'][1]-cube['a'][0]+1)*(cube['s'][1]-cube['s'][0]+1)

In [53]:
def follow_cube(cube: dict[str, tuple[int,int]], tag: str, wf: dict[str, list[tuple[None|tuple[str, str, int], str]]]) -> int:
    conds = wf[tag]
    s = 0
    for c in conds:
        if c[0] is None:
            if c[1] == 'R':
                s += 0
            elif c[1] == 'A':
                s += vol(cube)
            else:
                s += follow_cube(cube, c[1], wf)
        else:
            coord = c[0][0]
            val = c[0][2]
            if cube[coord][1] < val:
                if c[0][1] == -1:
                    if c[1] == 'R':
                        s += 0
                    elif c[1] == 'A':
                        s += vol(cube)
                    else:
                        s += follow_cube(cube, c[1], wf)
            elif cube[coord][0] > val:
                if c[0][1] == 1:
                    if c[1] == 'R':
                        s += 0
                    elif c[1] == 'A':
                        s += vol(cube)
                    else:
                        s += follow_cube(cube, c[1], wf)
            else:
                cube_c = cube.copy()
                if c[0][1] == 1:
                    cube_c[coord] = (val+1, cube[coord][1])
                    cube[coord] = (cube[coord][0], val)
                else:
                    cube_c[coord] = (cube[coord][0], val-1)
                    cube[coord] = (val, cube[coord][1])
                if c[1] == 'R':
                    s += 0
                elif c[1] == 'A':
                    s += vol(cube_c)
                else:
                    s += follow_cube(cube_c, c[1], wf)
    return s

In [56]:
def part2(text: str) -> int:
    wf, _ = parse_input(text)
    return follow_cube({i: (1, 4000) for i in 'xmas'}, 'in', wf)

In [57]:
part2(test)

167409079868000

In [58]:
with open('output2', 'wt') as f:
    f.write(str(part2(inp)))